In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# Adjusting Row Column Settings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

In [5]:
# Loading the Data Set

movie = pd.read_csv('./ml-25m/movies.csv')
rating = pd.read_csv('./ml-25m/ratings.csv')

In [6]:
# Merging movie and rating data sets

df = movie.merge(rating, how="left", on="movieId")

In [7]:
# Preliminary examination of the data set

def check_df(dataframe, head=5):
    print('##################### Shape #####################')
    print(dataframe.shape)
    print('##################### Types #####################')
    print(dataframe.dtypes)
    print('##################### Head #####################')
    print(dataframe.head(head))
    print('##################### Tail #####################')
    print(dataframe.tail(head))
    print('##################### NA #####################')
    print(dataframe.isnull().sum())
    print('##################### Quantiles #####################')
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

##################### Shape #####################
(25003471, 6)
##################### Types #####################
movieId        int64
title         object
genres        object
userId       float64
rating       float64
timestamp    float64
dtype: object
##################### Head #####################
   movieId             title                                       genres  userId  rating     timestamp
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy     2.0     3.5  1.141416e+09
1        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy     3.0     4.0  1.439472e+09
2        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy     4.0     3.0  1.573944e+09
3        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy     5.0     4.0  8.586259e+08
4        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy     8.0     4.0  8.904925e+08
##################### Tail #####################
          movieId       

In [8]:
# how many comments each film received.
comment_counts = pd.DataFrame(df["title"].value_counts())

In [9]:
comment_counts.head(20)

,title
Forrest Gump (1994),81491
"Shawshank Redemption, The (1994)",81482
Pulp Fiction (1994),79672
"Silence of the Lambs, The (1991)",74127
"Matrix, The (1999)",72674
Star Wars: Episode IV - A New Hope (1977),68717
Jurassic Park (1993),64144
Schindler's List (1993),60411
Braveheart (1995),59184
Fight Club (1999),58773


In [10]:
rare_movies = comment_counts[comment_counts["title"] <= 1000].index

In [11]:
rare_movies

Index(['Black Mass (2015)', 'Farinelli: il castrato (1994)', 'Darkest Hour (2017)', 'Stakeout (1987)', 'Corruptor, The (1999)', '47 Ronin (2013)', 'Princess and the Warrior, The (Krieger und die Kaiserin, Der) (2000)', 'Jane Austen's Mafia! (1998)', 'Crossroads (2002)', 'Bloodsport 2 (a.k.a. Bloodsport II: The Next Kumite) (1996)',
       ...
       'Nick Cannon: Stand Up, Don't Shoot (2017)', 'Lethal Angels (2006)', 'El cimarrón (2006)', 'White Rose Campus: Then Everybody Gets Raped (1982)', 'Lust in the Mummy's Tomb (2002)', 'Alive (2015)', 'The Friends (1994)', 'Woman Undone (1996)', 'Savage Abduction (1973)', 'Women of Devil's Island (1962)'],
      dtype='object', length=58535)

In [12]:
common_movies = df[~df["title"].isin(rare_movies)]

In [13]:
common_movies["title"].nunique()

3790

In [14]:
df["title"].nunique()

62325

In [15]:
# Creating User Movie Df
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")

In [16]:
user_movie_df.head()

title   'burbs, The (1989)  (500) Days of Summer (2009)  *batteries not included (1987)  ...And Justice for All (1979)  10 Cloverfield Lane (2016)  10 Things I Hate About You (1999)  10,000 BC (2008)  101 Dalmatians (1996)  101 Dalmatians (One Hundred and One Dalmatians) (1961)  102 Dalmatians (2000)  12 Angry Men (1957)  12 Years a Slave (2013)  127 Hours (2010)  13 Going on 30 (2004)  13th Warrior, The (1999)  1408 (2007)  15 Minutes (2001)  16 Blocks (2006)  17 Again (2009)  1984 (Nineteen Eighty-Four) (1984)  2 Days in the Valley (1996)  2 Fast 2 Furious (Fast and the Furious 2, The) (2003)  2 Guns (2013)  20,000 Leagues Under the Sea (1954)  200 Cigarettes (1999)  2001: A Space Odyssey (1968)  2010: The Year We Make Contact (1984)  2012 (2009)  2046 (2004)  21 (2008)  21 Grams (2003)  21 Jump Street (2012)  22 Jump Street (2014)  24 Hour Party People (2002)  25th Hour (2002)  27 Dresses (2008)  28 Days (2000)  28 Days Later (2002)  28 Weeks Later (2007)  3 Idiots (2009)  3-Iron (Bin-jip) (2004)  30 Days of Night (2007)  300 (2007)  3000 Miles to Graceland (2001)  300: Rise of an Empire (2014)  39 Steps, The (1935)  3:10 to Yuma (2007)  40 Days and 40 Nights (2002)  40-Year-Old Virgin, The (2005)  400 Blows, The (Les quatre cents coups) (1959)  42 Up (1998)  48 Hrs. (1982)  50 First Dates (2004)  50/50 (2011)  54 (1998)  6th Day, The (2000)  7th Voyage of Sinbad, The (1958)  8 1/2 (8½) (1963)  8 Heads in a Duffel Bag (1997)  8 Mile (2002)  8MM (1999)  9 (2009)  9 1/2 Weeks (Nine 1/2 Weeks) (1986)  A Million Ways to Die in the West (2014)  A Quiet Place (2018)  A Star Is Born (2018)  A-Team, The (2010)  A.I. Artificial Intelligence (2001)  AVP: Alien vs. Predator (2004)  AVPR: Aliens vs. Predator - Requiem (2007)  About Last Night... (1986)  About Schmidt (2002)  About Time (2013)  About a Boy (2002)  Abraham Lincoln: Vampire Hunter (2012)  Absent-Minded Professor, The (1961)  Absolute Power (1997)  Abyss, The (1989)  Accepted (2006)  Accidental Tourist, The (1988)  Accused, The (1988)  Ace Ventura: Pet Detective (1994)  Ace Ventura: When Nature Calls (1995)  Across the Universe (2007)  Adam's Rib (1949)  Adaptation (2002)  Addams Family Values (1993)  Addams Family, The (1991)  Addicted to Love (1997)  Adjustment Bureau, The (2011)  Adventureland (2009)  Adventures in Babysitting (1987)  Adventures of Baron Munchausen, The (1988)  Adventures of Buckaroo Banzai Across the 8th Dimension, The (1984)  Adventures of Milo and Otis, The (Koneko monogatari) (1986)  Adventures of Pinocchio, The (1996)  Adventures of Priscilla, Queen of the Desert, The (1994)  Adventures of Robin Hood, The (1938)  Adventures of Rocky and Bullwinkle, The (2000)  Adventures of Tintin, The (2011)  Aeon Flux (2005)  Affair to Remember, An (1957)  Affliction (1997)  African Queen, The (1951)  After Earth (2013)  After Hours (1985)  Age of Innocence, The (1993)  Agent Cody Banks (2003)  Agnes of God (1985)  Aguirre: The Wrath of God (Aguirre, der Zorn Gottes) (1972)  Air America (1990)  Air Bud (1997)  Air Force One (1997)  Airheads (1994)  Airplane II: The Sequel (1982)  Airplane! (1980)  Airport (1970)  Akira (1988)  Aladdin (1992)  Alexander (2004)  Alfie (2004)  Ali (2001)  Alice in Wonderland (1951)  Alice in Wonderland (2010)  Alien (1979)  Alien Nation (1988)  Alien: Covenant (2017)  Alien: Resurrection (1997)  Aliens (1986)  Alien³ (a.k.a. Alien 3) (1992)  Alive (1993)  All About Eve (1950)  All About My Mother (Todo sobre mi madre) (1999)  All Dogs Go to Heaven (1989)  All Dogs Go to Heaven 2 (1996)  All Quiet on the Western Front (1930)  All That Jazz (1979)  All the President's Men (1976)  Almost Famous (2000)  Along Came Polly (2004)  Along Came a Spider (2001)  Alphaville (Alphaville, une étrange aventure de Lemmy Caution) (1965)  Altered States (1980)  Alvin and the Chipmunks (2007)  Always (1989)  Amadeus (1984)  Amateur (1994)  Amazing Spider-Man, The (2012)  Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)  America's Sweethearts (2001

In [17]:
user_movie_df.shape

(162539, 3790)

In [18]:
user_movie_df.columns


Index([''burbs, The (1989)', '(500) Days of Summer (2009)', '*batteries not included (1987)', '...And Justice for All (1979)', '10 Cloverfield Lane (2016)', '10 Things I Hate About You (1999)', '10,000 BC (2008)', '101 Dalmatians (1996)', '101 Dalmatians (One Hundred and One Dalmatians) (1961)', '102 Dalmatians (2000)',
       ...
       'Zodiac (2007)', 'Zombieland (2009)', 'Zoolander (2001)', 'Zootopia (2016)', 'Zulu (1964)', '[REC] (2007)', 'eXistenZ (1999)', 'xXx (2002)', 'xXx: State of the Union (2005)', '¡Three Amigos! (1986)'],
      dtype='object', name='title', length=3790)

In [21]:
user_movie_df.head(1)

title   'burbs, The (1989)  (500) Days of Summer (2009)  *batteries not included (1987)  ...And Justice for All (1979)  10 Cloverfield Lane (2016)  10 Things I Hate About You (1999)  10,000 BC (2008)  101 Dalmatians (1996)  101 Dalmatians (One Hundred and One Dalmatians) (1961)  102 Dalmatians (2000)  12 Angry Men (1957)  12 Years a Slave (2013)  127 Hours (2010)  13 Going on 30 (2004)  13th Warrior, The (1999)  1408 (2007)  15 Minutes (2001)  16 Blocks (2006)  17 Again (2009)  1984 (Nineteen Eighty-Four) (1984)  2 Days in the Valley (1996)  2 Fast 2 Furious (Fast and the Furious 2, The) (2003)  2 Guns (2013)  20,000 Leagues Under the Sea (1954)  200 Cigarettes (1999)  2001: A Space Odyssey (1968)  2010: The Year We Make Contact (1984)  2012 (2009)  2046 (2004)  21 (2008)  21 Grams (2003)  21 Jump Street (2012)  22 Jump Street (2014)  24 Hour Party People (2002)  25th Hour (2002)  27 Dresses (2008)  28 Days (2000)  28 Days Later (2002)  28 Weeks Later (2007)  3 Idiots (2009)  3-Iron (Bin-jip) (2004)  30 Days of Night (2007)  300 (2007)  3000 Miles to Graceland (2001)  300: Rise of an Empire (2014)  39 Steps, The (1935)  3:10 to Yuma (2007)  40 Days and 40 Nights (2002)  40-Year-Old Virgin, The (2005)  400 Blows, The (Les quatre cents coups) (1959)  42 Up (1998)  48 Hrs. (1982)  50 First Dates (2004)  50/50 (2011)  54 (1998)  6th Day, The (2000)  7th Voyage of Sinbad, The (1958)  8 1/2 (8½) (1963)  8 Heads in a Duffel Bag (1997)  8 Mile (2002)  8MM (1999)  9 (2009)  9 1/2 Weeks (Nine 1/2 Weeks) (1986)  A Million Ways to Die in the West (2014)  A Quiet Place (2018)  A Star Is Born (2018)  A-Team, The (2010)  A.I. Artificial Intelligence (2001)  AVP: Alien vs. Predator (2004)  AVPR: Aliens vs. Predator - Requiem (2007)  About Last Night... (1986)  About Schmidt (2002)  About Time (2013)  About a Boy (2002)  Abraham Lincoln: Vampire Hunter (2012)  Absent-Minded Professor, The (1961)  Absolute Power (1997)  Abyss, The (1989)  Accepted (2006)  Accidental Tourist, The (1988)  Accused, The (1988)  Ace Ventura: Pet Detective (1994)  Ace Ventura: When Nature Calls (1995)  Across the Universe (2007)  Adam's Rib (1949)  Adaptation (2002)  Addams Family Values (1993)  Addams Family, The (1991)  Addicted to Love (1997)  Adjustment Bureau, The (2011)  Adventureland (2009)  Adventures in Babysitting (1987)  Adventures of Baron Munchausen, The (1988)  Adventures of Buckaroo Banzai Across the 8th Dimension, The (1984)  Adventures of Milo and Otis, The (Koneko monogatari) (1986)  Adventures of Pinocchio, The (1996)  Adventures of Priscilla, Queen of the Desert, The (1994)  Adventures of Robin Hood, The (1938)  Adventures of Rocky and Bullwinkle, The (2000)  Adventures of Tintin, The (2011)  Aeon Flux (2005)  Affair to Remember, An (1957)  Affliction (1997)  African Queen, The (1951)  After Earth (2013)  After Hours (1985)  Age of Innocence, The (1993)  Agent Cody Banks (2003)  Agnes of God (1985)  Aguirre: The Wrath of God (Aguirre, der Zorn Gottes) (1972)  Air America (1990)  Air Bud (1997)  Air Force One (1997)  Airheads (1994)  Airplane II: The Sequel (1982)  Airplane! (1980)  Airport (1970)  Akira (1988)  Aladdin (1992)  Alexander (2004)  Alfie (2004)  Ali (2001)  Alice in Wonderland (1951)  Alice in Wonderland (2010)  Alien (1979)  Alien Nation (1988)  Alien: Covenant (2017)  Alien: Resurrection (1997)  Aliens (1986)  Alien³ (a.k.a. Alien 3) (1992)  Alive (1993)  All About Eve (1950)  All About My Mother (Todo sobre mi madre) (1999)  All Dogs Go to Heaven (1989)  All Dogs Go to Heaven 2 (1996)  All Quiet on the Western Front (1930)  All That Jazz (1979)  All the President's Men (1976)  Almost Famous (2000)  Along Came Polly (2004)  Along Came a Spider (2001)  Alphaville (Alphaville, une étrange aventure de Lemmy Caution) (1965)  Altered States (1980)  Alvin and the Chipmunks (2007)  Always (1989)  Amadeus (1984)  Amateur (1994)  Amazing Spider-Man, The (2012)  Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)  America's Sweethearts (2001

In [50]:
def checkMoviePresentInDB(movie_name,user_movie_df):
    result_array = np.core.defchararray.find(list(user_movie_df.columns), movie_name) != -1
    result = list(user_movie_df.columns[result_array])
    if len(result)>0:
        return (True,False,result)
    else:
        return (False,False,result)
    
def item_based_recommender(movie_name, user_movie_df,topN=10):
    if(~isinstance(topN,int)):
        topN = 10
    try:
        movie_name = user_movie_df[movie_name]
        result = user_movie_df.corrwith(movie_name).sort_values(ascending=False).iloc[1:topN+1]
        return (True,True,result)
    except KeyError as err:
        return checkMoviePresentInDB(movie_name,user_movie_df)
    

In [60]:
response = item_based_recommender("Live Free or Die Hard (2007)",user_movie_df,"a")
if(response[0]==True & response[1] == True): #Response when recommend Movie list is provided in Response
    print("Recommended Top 10 Movies are :")
    print(response[2])
elif(response[0]==True and response[1]==False): #Response when multiple movies with same title are seen in Movie DB
    print("Movie Requested has multiple similar titles, please use any one of the following")
    print("\n".join(response[2]))
else:                                          #Response when no match is found in movie DB for the title requested by User
    print("Movie not found in movie DB")
    

Recommended Top 10 Movies are :
title
Good Day to Die Hard, A (2013)                         0.605174
Widows' Peak (1994)                                    0.588179
Fast & Furious (Fast and the Furious 4, The) (2009)    0.580841
Non-Stop (2014)                                        0.577438
Eagle Eye (2008)                                       0.568900
Sorcerer's Apprentice, The (2010)                      0.565375
The Expendables 3 (2014)                               0.564782
Death Race (2008)                                      0.563718
Taken 2 (2012)                                         0.563315
San Andreas (2015)                                     0.556715
dtype: float64
